In [1]:
import numpy as np
from IPython.display import clear_output, display
from pydrake.all import (AbstractValue, AddMultibodyPlantSceneGraph,
                         DiagramBuilder, JointSliders, LeafSystem,
                         MeshcatVisualizer, Parser, RigidTransform,
                         RollPitchYaw, StartMeshcat)

from manipulation import FindResource, running_as_notebook
from manipulation.scenarios import AddMultibodyTriad, AddPackagePaths

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
pose_translation = ""
pose_rotation = ""

class PrintPose(LeafSystem): #inheriting the leafs
    def __init__(self, body_index):
        LeafSystem.__init__(self)
        self._body_index = body_index
        self.DeclareAbstractInputPort("body_poses",
                                    AbstractValue.Make([RigidTransform()]))
        self.DeclareForcedPublishEvent(self.Publish)
        self.system_context = self.AllocateContext()
        print("system context")
        print(self.system_context)

    def Publish(self, context):
        print("context")
        print(context)
        pose = self.get_input_port().Eval(context)[self._body_index]
        clear_output(wait=False)
        pose_translation =  np.array2string(
            pose.translation(), formatter={
                'float': lambda x: "{:3.2f}".format(x)})
        print("gripper position (m): " + pose_translation)
        pose_rotation = np.array2string(RollPitchYaw(pose.rotation()).vector(),
                         formatter={'float': lambda x: "{:3.2f}".format(x)})
        print("gripper roll-pitch-yaw (rad):" + pose_rotation)

def gripper_forward_kinematics_example():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0)
    parser = Parser(plant)
    AddPackagePaths(parser)
    parser.AddAllModelsFromFile(FindResource("models/iiwa_and_wsg.dmd.yaml"))
    plant.Finalize()

    # Draw the frames
    for body_name in ["iiwa_link_1", "iiwa_link_2", "iiwa_link_3", "iiwa_link_4", "iiwa_link_5", "iiwa_link_6", "iiwa_link_7", "body"]:
        AddMultibodyTriad(plant.GetFrameByName(body_name), scene_graph)

    meshcat.Delete()
    meshcat.DeleteAddedControls()
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph.get_query_output_port(), meshcat)

    wsg = plant.GetModelInstanceByName("wsg")
    gripper = plant.GetBodyByName("body", wsg)
    ppob = PrintPose(gripper.index())
    print_pose = builder.AddSystem(ppob)
    builder.Connect(plant.get_body_poses_output_port(),
                    print_pose.get_input_port())

    default_interactive_timeout = None if running_as_notebook else 1.0
    sliders = builder.AddSystem(JointSliders(meshcat, plant))
    diagram = builder.Build()
    
    
    sliders.Run(diagram, default_interactive_timeout)
    meshcat.DeleteAddedControls()
        


In [ ]:
gripper_forward_kinematics_example()

gripper position (m): [0.08 -0.38 0.91]
gripper roll-pitch-yaw (rad):[-2.39 0.31 -0.14]
context
::_::__main__.PrintPose@00000000034cbd30 Context
-------------------------------------------------
Time: 0



In [ ]:
gripper_forward_kinematics_example()

In [ ]:
class ninja:
    def __init__(self,style):
        self.ninja_style = style



In [ ]:
n = ninja(2)

In [ ]:
n.ninja_style